In [101]:
# importing useful libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [102]:
#Step 2: Scraping Wikipedia page to obtain postal codes data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
table=soup.find('table')

In [103]:
#Step 3: Selecting the three columns of data we want: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [104]:
#Step 4: Searching for all the postcode, borough, neighborhood data
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [105]:
#Step 5: Removing data where Borough is "Not assigned"
df = df[df.Borough != 'Not assigned']

In [106]:
#Step 6: Combining neighbourhoods with the same postalcode

df1=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df1=df1.reset_index(drop=False)
df1.rename(columns={'Neighborhood':'Neighborhoods'},inplace=True)

df2 = pd.merge(df, df1, on='Postalcode')
df2.drop(['Neighborhood'],axis=1,inplace=True)
df2.drop_duplicates(inplace=True)
df2.rename(columns={'Neighborhoods':'Neighborhood'},inplace=True)
            


### Question-2

### To utilize the Foursquare location data  to get the latitude and the longitude coordinates of each neighborhood around Toron

In [107]:
#Step 1: Importing CSV with the latitudes and longitudes
latlon = pd.read_csv('https://cocl.us/Geospatial_data')
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [114]:
#Step 2: Merging data for neighbourhoods in Canada
latlon.rename(columns={'Postal Code':'Postalcode'},inplace=True)
Toronto_df  = pd.merge(latlon, df2, on='Postalcode')
Toronto_df.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [115]:
#reordering dataframe
Toronto_df =Toronto_df [['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
Toronto_df .head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [116]:
Toronto_df .shape

(103, 5)